<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/cnn_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [53]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [55]:
class ImageDataLoader(Dataset):
    def __init__(self, dir_=None):
        self.data_df = pd.read_csv('gdrive/MyDrive/data.csv')
        self.dataset_len = len(self.data_df) # read the number of len of your csv files
    def __getitem__(self, idx):
        # load the next image
        f_name_t = self.data_df['Filename'][idx]
        f_name_tp1 = self.data_df['Filename'][idx+1]
        label = self.data_df['Label'][idx]
        img_t = torchvision.io.read_image('gdrive/MyDrive/trainingData/{}'.format(f_name_t))
        img_tp1 = torchvision.io.read_image('gdrive/MyDrive/trainingData/{}'.format(f_name_tp1))
        img_t = img_t.float().div_(255.0)
        img_tp1 = img_tp1.float().div_(255.0)
        return img_t, img_tp1, label
    def __len__(self):
        return self.dataset_len - 1

In [56]:
dataset = ImageDataLoader()
dataloader = DataLoader(dataset, shuffle=False, batch_size=8)

In [57]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 1, 1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 1, 1),
            nn.ReLU(),
            nn.Flatten()
        )
    def forward(self, x):
        state = self.cnn(x)
        # print('size of the state after CNN ',state.size())
        return state

In [58]:
encoder = Encoder()

In [59]:
num_epochs = 10
learning_rate = 0.001
input_size = 131072
output_size = 1
sequence_length = 2
hidden_size = 128
num_layers = 2

In [60]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)  
        # out = out[:, -1]
        out = self.fc(out)
        return out

In [61]:
model = RNN(input_size, hidden_size, num_layers, output_size)

In [62]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
n_total_steps = len(dataloader)
for epoch in range(num_epochs):
    for i, (image1, image2, label) in enumerate(dataloader):
        image1, image2, label = next(iter(dataloader))
        output = encoder(image1)
        output1 = encoder(image2)
        output = output.reshape(8,1,-1)
        output1 = output1.reshape(8,1,-1)
        seq = torch.cat((output, output1), dim=1)
        label = (label.float())

        # Forward pass
        outputs = model(seq)
        loss = criterion(outputs[:,-1], label)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}] | Step [{i+1}/{n_total_steps}] | Loss: {loss.item():.4f}')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10] | Step [10/32] | Loss: 0.2603
Epoch [1/10] | Step [20/32] | Loss: 0.0715
Epoch [1/10] | Step [30/32] | Loss: 0.0348
Epoch [2/10] | Step [10/32] | Loss: 0.0087
Epoch [2/10] | Step [20/32] | Loss: 0.0040
Epoch [2/10] | Step [30/32] | Loss: 0.0015
Epoch [3/10] | Step [10/32] | Loss: 0.0003
Epoch [3/10] | Step [20/32] | Loss: 0.0001
Epoch [3/10] | Step [30/32] | Loss: 0.0000
Epoch [4/10] | Step [10/32] | Loss: 0.0000
Epoch [4/10] | Step [20/32] | Loss: 0.0000
Epoch [4/10] | Step [30/32] | Loss: 0.0000
Epoch [5/10] | Step [10/32] | Loss: 0.0000
Epoch [5/10] | Step [20/32] | Loss: 0.0000
Epoch [5/10] | Step [30/32] | Loss: 0.0000
Epoch [6/10] | Step [10/32] | Loss: 0.0000
Epoch [6/10] | Step [20/32] | Loss: 0.0000
Epoch [6/10] | Step [30/32] | Loss: 0.0000
Epoch [7/10] | Step [10/32] | Loss: 0.0000
Epoch [7/10] | Step [20/32] | Loss: 0.0000
Epoch [7/10] | Step [30/32] | Loss: 0.0000
Epoch [8/10] | Step [10/32] | Loss: 0.0000
Epoch [8/10] | Step [20/32] | Loss: 0.0000
Epoch [8/10